In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("FinalDataset.csv")

In [4]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

In [6]:
df = df.drop(columns=['Unnamed: 0'])

In [8]:
# filter year 2022
df_2022 = df.loc['2022']

In [9]:
# temperature column name
temp_col = 'TempF'

In [10]:
# columns except temperature
cols_except_temp = [c for c in df_2022.columns if c != temp_col]

In [11]:
# daily averages (except temperature)
daily_avg = df_2022[cols_except_temp].resample('D').mean()

In [12]:
# daily temperature stats
daily_temp = df_2022[temp_col].resample('D').agg(
    max_temp='max',
    min_temp='min'
)

In [13]:
# combine
daily_df = pd.concat([daily_avg, daily_temp], axis=1)

In [14]:
daily_df.head()

,LMP,IMF1,IMF2,IMF3,IMF4,IMF5,IMF6,IMF7,IMF8,IMF9,...,hour,week_end,day_of_month,mid_week,Day sin,Day cos,Year sin,Year cos,max_temp,min_temp
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,34.497917,36.950558,-0.245150,0.010785,-0.039383,-0.009603,-0.000239,0.003572,-0.000948,-0.002703,...,11.5,1.0,1.0,0.0,-3.926281e-13,3.920799e-13,0.014951,0.999876,-0.9,-11.0
2022-01-02,40.301250,38.173651,-0.142536,-0.002492,-0.015850,-0.036455,-0.003100,-0.011704,-0.014835,0.001162,...,11.5,1.0,2.0,0.0,2.137480e-13,-2.116640e-13,0.032149,0.999471,3.0,-14.1
2022-01-03,40.242083,36.089147,0.454951,0.038163,0.059690,-0.004728,-0.003024,0.005284,0.003943,0.006156,...,11.5,0.0,3.0,0.0,2.135352e-13,-2.137457e-13,0.049337,0.998770,25.0,3.0
2022-01-04,24.118333,32.168110,-0.351955,-0.083060,0.000399,-0.018603,-0.012894,-0.007401,-0.001073,-0.002026,...,11.5,0.0,4.0,1.0,-3.923690e-13,3.937498e-13,0.066510,0.997773,30.9,16.0
2022-01-05,27.630833,35.588176,-0.221203,0.043148,-0.064583,-0.033133,0.005629,-0.004904,-0.017024,-0.009489,...,11.5,0.0,5.0,1.0,2.120827e-13,-2.137457e-13,0.083664,0.996482,21.0,3.0


In [15]:
daily_df = daily_df.drop(columns=['hour'])

In [16]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 338 entries, 2022-01-01 to 2022-12-04
Freq: D
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LMP           338 non-null    float64
 1   IMF1          338 non-null    float64
 2   IMF2          338 non-null    float64
 3   IMF3          338 non-null    float64
 4   IMF4          338 non-null    float64
 5   IMF5          338 non-null    float64
 6   IMF6          338 non-null    float64
 7   IMF7          338 non-null    float64
 8   IMF8          338 non-null    float64
 9   IMF9          338 non-null    float64
 10  IMF10         338 non-null    float64
 11  IMF11         338 non-null    float64
 12  IMF12         338 non-null    float64
 13  WS(mph)       338 non-null    float64
 14  week_end      338 non-null    float64
 15  day_of_month  338 non-null    float64
 16  mid_week      338 non-null    float64
 17  Day sin       338 non-null    float64
 18  Day

In [17]:
daily_df['week_end'] = (daily_df['week_end'] > 0.5).astype(int)
daily_df['mid_week'] = (daily_df['mid_week'] > 0.5).astype(int)

In [18]:
daily_df.to_csv('daily_df.csv')

In [20]:
df2 = pd.read_csv("us_daily_energy_2022.csv")

df3 = pd.read_csv("us_daily_generation_different_sources_2022.csv")

In [22]:
# Prepare df2 and df3 by setting the 'period' column as a datetime index
df2['period'] = pd.to_datetime(df2['period'])
df2 = df2.set_index('period')

df3['period'] = pd.to_datetime(df3['period'])
# Note: df3 has multiple rows per date (fuel types).
# If you want to join it, you should pivot it first so each fuel type is a column.
df3_pivot = df3.pivot(index='period', columns='fuel', values='generation_MWh')

# Now join the dataframes
merged_df = daily_df.join(df2, how='left').join(df3_pivot, how='left')
merged_df.head()

,LMP,IMF1,IMF2,IMF3,IMF4,IMF5,IMF6,IMF7,IMF8,IMF9,...,Net generation,Total interchange,Coal,Hydro,Natural Gas,Nuclear,Other,Petroleum,Solar,Wind
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,34.497917,36.950558,-0.245150,0.010785,-0.039383,-0.009603,-0.000239,0.003572,-0.000948,-0.002703,...,29383223,-177129,5773854,2424090,8630241,6915615,541869,62073,576708,4457331
2022-01-02,40.301250,38.173651,-0.142536,-0.002492,-0.015850,-0.036455,-0.003100,-0.011704,-0.014835,0.001162,...,31237195,-290674,6689943,2398782,10963404,6915756,536574,64398,616311,3024690
2022-01-03,40.242083,36.089147,0.454951,0.038163,0.059690,-0.004728,-0.003024,0.005284,0.003943,0.006156,...,34926384,-318716,7357572,2628471,13300701,6951003,596037,132255,660924,3295071
2022-01-04,24.118333,32.168110,-0.351955,-0.083060,0.000399,-0.018603,-0.012894,-0.007401,-0.001073,-0.002026,...,35622266,-155446,7070958,2580795,12196374,6960789,636387,75651,662832,5424765
2022-01-05,27.630833,35.588176,-0.221203,0.043148,-0.064583,-0.033133,0.005629,-0.004904,-0.017024,-0.009489,...,34262977,-128312,7162800,2437491,11525235,6930051,615591,60951,641382,4855872


In [23]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 338 entries, 2022-01-01 to 2022-12-04
Freq: D
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   LMP                        338 non-null    float64
 1   IMF1                       338 non-null    float64
 2   IMF2                       338 non-null    float64
 3   IMF3                       338 non-null    float64
 4   IMF4                       338 non-null    float64
 5   IMF5                       338 non-null    float64
 6   IMF6                       338 non-null    float64
 7   IMF7                       338 non-null    float64
 8   IMF8                       338 non-null    float64
 9   IMF9                       338 non-null    float64
 10  IMF10                      338 non-null    float64
 11  IMF11                      338 non-null    float64
 12  IMF12                      338 non-null    float64
 13  WS(mph)                

In [24]:
merged_df.to_csv("Dataset.csv")